In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
import openpyxl
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
columns = ["content"]
df = pd.read_excel(r'C:\Users\Hp\TUGAS AKHIR 2\TWEET_INDIHOME\data mentah\crawlingtweet_indihome_1.xlsx', usecols=columns)
df.head(2000)

,content
0,"@LuthfiAuliaFaza Kak Luthfi, perihal konpensas..."
1,Min @IndiHome Knapa begini ya? https://t.co/6R...
2,"indihome taiii, flop banget anj. gue muk mulun..."
3,@IndiHome mohon untuk cek dm ya
4,@IndiHome Cara nyambungin LAN ke ps4 gimana ka...
...,...
1995,"Di rumah juga pakai @IndiHome, kacau bener kua..."
1996,@NuriHandayani2 Hi Kak Nuri. Kalo akses Intern...
1997,@viyakkk Halo Sobat. Boleh kasih tau detail yg...
1998,@HizbutTembakau @kawananraja @IndiHome @Telkom...


In [3]:
print('Dataset size:', df.shape)
print('Columns are:', df.columns)

Dataset size: (3000, 1)
Columns are: Index(['content'], dtype='object')


In [4]:
df.content.duplicated().sum()

22

In [5]:
df.drop_duplicates(subset="content", keep=False, inplace=True)
df.head(3000)

,content
0,"@LuthfiAuliaFaza Kak Luthfi, perihal konpensas..."
1,Min @IndiHome Knapa begini ya? https://t.co/6R...
2,"indihome taiii, flop banget anj. gue muk mulun..."
3,@IndiHome mohon untuk cek dm ya
4,@IndiHome Cara nyambungin LAN ke ps4 gimana ka...
...,...
2995,@IndiHome Ya min
2996,@IndiHome Tanggal 31 berarti besok dong ya? Ka...
2997,"@derinovember Kak Adhika, maaf ya internetanny..."
2998,ini indihome kenapa bermasalah mulu sih_-


In [6]:
df.content.duplicated().sum()

0

In [7]:
#case folding
df['content'] = df['content'].str.lower()
df.head(2000)

,content
0,"@luthfiauliafaza kak luthfi, perihal konpensas..."
1,min @indihome knapa begini ya? https://t.co/6r...
2,"indihome taiii, flop banget anj. gue muk mulun..."
3,@indihome mohon untuk cek dm ya
4,@indihome cara nyambungin lan ke ps4 gimana ka...
...,...
2023,"@mnyta_ siap kak adiet, mohon berkenan menungg..."
2024,"@alfinjauhari siang kak alfin, perihal keluhan..."
2025,"@zummyyummy hai kak zummy, untuk informasi fup..."
2026,@indihome @hizbuttembakau @telkomcare kak kimc...


In [8]:
#cleaning text
import nltk
import string
import re
nltk.download('punkt')

#import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
#proses cleaning text
def remove_special_tweet(text):
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", "", str(text)).split())
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r'\b\w{1,2}\b', ' ', str(text))
    text = re.sub(r'\s\s+',' ',  str(text))
    text = re.sub(r'\d+', ' ', str(text))
    text = re.sub(r'\b[a-zA-Z]\b', ' ', str(text))
    text = re.sub(r'/(?:https?|ftp):\/\/[\n\S]+/g, ''', ' ', str(text))
    text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', str(text))
    text = re.sub(r'[0]', r' ', str(text))
    return text
    return df
df['cleaning_tweet'] = df['content'].apply(remove_special_tweet)
df.head(3000)

,content,cleaning_tweet
0,"@luthfiauliafaza kak luthfi, perihal konpensas...",kak luthfi perihal konpensasi nya sesuaikan de...
1,min @indihome knapa begini ya? https://t.co/6r...,min knapa begini
2,"indihome taiii, flop banget anj. gue muk mulun...",indihome taiii flop banget anj gue muk mulung ...
3,@indihome mohon untuk cek dm ya,mohon untuk cek
4,@indihome cara nyambungin lan ke ps4 gimana ka...,cara nyambungin lan ps gimana kak gagal terus
...,...,...
2995,@indihome ya min,min
2996,@indihome tanggal 31 berarti besok dong ya? ka...,tanggal berarti besok dong kalau hari ini bisa...
2997,"@derinovember kak adhika, maaf ya internetanny...",kak adhika maaf internetannya terganggu dan su...
2998,ini indihome kenapa bermasalah mulu sih_-,ini indihome kenapa bermasalah mulu sih


In [10]:
#proses tokenisasi
def tokenisasi(text):
    text = re.split('\W+', str(text))
    return text
    return df
df['tokenisasi'] = df['cleaning_tweet'].apply(tokenisasi)
df.head(2000)

,content,cleaning_tweet,tokenisasi
0,"@luthfiauliafaza kak luthfi, perihal konpensas...",kak luthfi perihal konpensasi nya sesuaikan de...,"[kak, luthfi, perihal, konpensasi, nya, sesuai..."
1,min @indihome knapa begini ya? https://t.co/6r...,min knapa begini,"[min, knapa, begini, ]"
2,"indihome taiii, flop banget anj. gue muk mulun...",indihome taiii flop banget anj gue muk mulung ...,"[indihome, taiii, flop, banget, anj, gue, muk,..."
3,@indihome mohon untuk cek dm ya,mohon untuk cek,"[mohon, untuk, cek, ]"
4,@indihome cara nyambungin lan ke ps4 gimana ka...,cara nyambungin lan ps gimana kak gagal terus,"[cara, nyambungin, lan, ps, gimana, kak, gagal..."
...,...,...,...
2023,"@mnyta_ siap kak adiet, mohon berkenan menungg...",siap kak adiet mohon berkenan menunggu progre...,"[, siap, kak, adiet, mohon, berkenan, menunggu..."
2024,"@alfinjauhari siang kak alfin, perihal keluhan...",siang kak alfin perihal keluhannya sudah diban...,"[siang, kak, alfin, perihal, keluhannya, sudah..."
2025,"@zummyyummy hai kak zummy, untuk informasi fup...",hai kak zummy untuk informasi fup nya silakan ...,"[hai, kak, zummy, untuk, informasi, fup, nya, ..."
2026,@indihome @hizbuttembakau @telkomcare kak kimc...,kak kimcil apakah sudah jelas terimakasih bonan,"[kak, kimcil, apakah, sudah, jelas, terimakasi..."


In [11]:
#import nltk stopword
from nltk.corpus import stopwords

#proses stopword
stopword = nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo','kalo', 'amp', 'biar', 'bikin', 'bilang','gak', 'ga', 'krn', 'nya', 
                 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'tai', 'juga', 'utk', 'ya','jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe',
                 'pen', 'u', 'nan', 'loh', 'rt', 'kudu', 'mks', 'wkwk', 'an', 'lah', 'dah', 'asu', 'mjb', 'haha',
                 'yah', 'lha', 'lah', 'hihi', 'ih', 'anjir', 'bgt', 'pake', 'anj', 'bgst', 'nya'])
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    return df
df['stopword'] = df['tokenisasi'].apply(remove_stopwords)
df.head(3000)

,content,cleaning_tweet,tokenisasi,stopword
0,"@luthfiauliafaza kak luthfi, perihal konpensas...",kak luthfi perihal konpensasi nya sesuaikan de...,"[kak, luthfi, perihal, konpensasi, nya, sesuai...","[kak, luthfi, perihal, konpensasi, sesuaikan, ..."
1,min @indihome knapa begini ya? https://t.co/6r...,min knapa begini,"[min, knapa, begini, ]","[min, knapa, ]"
2,"indihome taiii, flop banget anj. gue muk mulun...",indihome taiii flop banget anj gue muk mulung ...,"[indihome, taiii, flop, banget, anj, gue, muk,...","[indihome, taiii, flop, banget, gue, muk, mulu..."
3,@indihome mohon untuk cek dm ya,mohon untuk cek,"[mohon, untuk, cek, ]","[mohon, cek, ]"
4,@indihome cara nyambungin lan ke ps4 gimana ka...,cara nyambungin lan ps gimana kak gagal terus,"[cara, nyambungin, lan, ps, gimana, kak, gagal...","[nyambungin, lan, ps, gimana, kak, gagal]"
...,...,...,...,...
2995,@indihome ya min,min,"[, min]","[, min]"
2996,@indihome tanggal 31 berarti besok dong ya? ka...,tanggal berarti besok dong kalau hari ini bisa...,"[tanggal, berarti, besok, dong, kalau, hari, i...","[tanggal, besok, nggak, ]"
2997,"@derinovember kak adhika, maaf ya internetanny...",kak adhika maaf internetannya terganggu dan su...,"[kak, adhika, maaf, internetannya, terganggu, ...","[kak, adhika, maaf, internetannya, terganggu, ..."
2998,ini indihome kenapa bermasalah mulu sih_-,ini indihome kenapa bermasalah mulu sih,"[ini, indihome, kenapa, bermasalah, mulu, sih, ]","[indihome, bermasalah, mulu, ]"


In [12]:
#stemming
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]
    return df

df['stemming'] = df['stopword'].swifter.apply(get_stemmed_term)
df.head(3000)

4753
------------------------
kak : kak
luthfi : luthfi
perihal : perihal
konpensasi : konpensasi
sesuaikan : sesuai
gangguan : ganggu
disesuaikan : sesuai
kebijakan : bijak
plasa : plasa
telkom : telkom
diupayakan : upaya
secepatnya : cepat
normal : normal
terima : terima
kasih : kasih
nufi : nufi
min : min
knapa : knapa
 : 
indihome : indihome
taiii : taiii
flop : flop
banget : banget
gue : gue
muk : muk
mulung : mulung
loading : loading
iklan : iklan
mohon : mohon
cek : cek
nyambungin : nyambungin
lan : lan
ps : ps
gimana : gimana
gagal : gagal
trian : tri
halo : halo
sobat : sobat
koneksi : koneksi
los : los
merah : merah
langkah : langkah
pastikan : pasti
kabel : kabel
patch : patch
cord : cord
berwarna : warna
hitam : hitam
kuning : kuning
ujung : ujung
biru : biru
tertancap : tancap
terpasang : pasang
kencang : kencang
silakan : sila
konfirmasikan : konfirmasi
kembal : kembal
jual : jual
order : order
spotify : spotify
murah : murah
aktivasi : aktivasi
perpanjangan : panjang
pak

Pandas Apply:   0%|          | 0/2963 [00:00<?, ?it/s]

,content,cleaning_tweet,tokenisasi,stopword,stemming
0,"@luthfiauliafaza kak luthfi, perihal konpensas...",kak luthfi perihal konpensasi nya sesuaikan de...,"[kak, luthfi, perihal, konpensasi, nya, sesuai...","[kak, luthfi, perihal, konpensasi, sesuaikan, ...","[kak, luthfi, perihal, konpensasi, sesuai, gan..."
1,min @indihome knapa begini ya? https://t.co/6r...,min knapa begini,"[min, knapa, begini, ]","[min, knapa, ]","[min, knapa, ]"
2,"indihome taiii, flop banget anj. gue muk mulun...",indihome taiii flop banget anj gue muk mulung ...,"[indihome, taiii, flop, banget, anj, gue, muk,...","[indihome, taiii, flop, banget, gue, muk, mulu...","[indihome, taiii, flop, banget, gue, muk, mulu..."
3,@indihome mohon untuk cek dm ya,mohon untuk cek,"[mohon, untuk, cek, ]","[mohon, cek, ]","[mohon, cek, ]"
4,@indihome cara nyambungin lan ke ps4 gimana ka...,cara nyambungin lan ps gimana kak gagal terus,"[cara, nyambungin, lan, ps, gimana, kak, gagal...","[nyambungin, lan, ps, gimana, kak, gagal]","[nyambungin, lan, ps, gimana, kak, gagal]"
...,...,...,...,...,...
2995,@indihome ya min,min,"[, min]","[, min]","[, min]"
2996,@indihome tanggal 31 berarti besok dong ya? ka...,tanggal berarti besok dong kalau hari ini bisa...,"[tanggal, berarti, besok, dong, kalau, hari, i...","[tanggal, besok, nggak, ]","[tanggal, besok, nggak, ]"
2997,"@derinovember kak adhika, maaf ya internetanny...",kak adhika maaf internetannya terganggu dan su...,"[kak, adhika, maaf, internetannya, terganggu, ...","[kak, adhika, maaf, internetannya, terganggu, ...","[kak, adhika, maaf, internetannya, ganggu, sun..."
2998,ini indihome kenapa bermasalah mulu sih_-,ini indihome kenapa bermasalah mulu sih,"[ini, indihome, kenapa, bermasalah, mulu, sih, ]","[indihome, bermasalah, mulu, ]","[indihome, masalah, mulu, ]"


In [23]:
Final = df[['stemming']]
Final = Final.rename(columns={'stemming':'Tweets'})

In [24]:
Final

,Tweets
0,kakluthfiperihalkonpensasisesuaiganggusesuaibi...
1,minknapa
2,indihometaiiiflopbangetguemukmulungloadingiklan
3,mohoncek
4,nyambunginlanpsgimanakakgagal
...,...
2995,min
2996,tanggalbesoknggak
2997,kakadhikamaafinternetannyaganggusunnyrefreshja...
2998,indihomemasalahmulu
